In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader

In [2]:
train = pd.read_csv("/kaggle/input/llm-classification-finetuning/train.csv", index_col=0)

In [3]:
train.head()

,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
id,,,,,,,,
30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0


In [4]:
train["prompt"] = train.prompt.map(lambda x: eval(x)[0])
train["response_a"] = train.response_a.map(lambda x: eval(x.replace("null","''"))[0])
train["response_b"] = train.response_b.map(lambda x: eval(x.replace("null", "''"))[0])

In [5]:
train["class_name"] = train[["winner_model_a", "winner_model_b" , "winner_tie"]].idxmax(axis=1)
train["class_label"] = train.class_name.map({'winner_model_a':0, 'winner_model_b': 1, 'winner_tie' : 2})


train.head()

,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,class_name,class_label
id,,,,,,,,,,
30192,gpt-4-1106-preview,gpt-4-0613,Is it morally right to try to have a certain p...,The question of whether it is morally right to...,"As an AI, I don't have personal beliefs or opi...",1,0,0,winner_model_a,0
53567,koala-13b,gpt-4-0613,What is the difference between marriage licens...,A marriage license is a legal document that al...,A marriage license and a marriage certificate ...,0,1,0,winner_model_b,1
65089,gpt-3.5-turbo-0613,mistral-medium,explain function calling. how would you call a...,Function calling is the process of invoking or...,Function calling is the process of invoking a ...,0,0,1,winner_tie,2
96401,llama-2-13b-chat,mistral-7b-instruct,How can I create a test set for a very rare ca...,Creating a test set for a very rare category c...,When building a classifier for a very rare cat...,1,0,0,winner_model_a,0
198779,koala-13b,gpt-3.5-turbo-0314,What is the best way to travel from Tel-Aviv t...,The best way to travel from Tel Aviv to Jerusa...,The best way to travel from Tel-Aviv to Jerusa...,0,1,0,winner_model_b,1


In [6]:
def make_pairs(row):
    row["encode_fail"] = False
    try:
        prompt = row.prompt.encode("utf-8").decode("utf-8")
    except Exception:
        prompt = ""
        row["encode_fail"] = True

    try:
        response_a = row.response_a.encode("utf-8").decode("utf-8")
    except Exception:
        response_a = ""
        row["encode_fail"] = True

    try:
        response_b = row.response_b.encode("utf-8").decode("utf-8")
    except Exception:
        response_b = ""
        row["encode_fail"] = True
        
    row['options'] = [f"Prompt: {prompt}\n\nResponse: {response_a}",
                      f"Prompt: {prompt}\n\nResponse: {response_b}"
                     ]
    return row

In [7]:
train = train.apply(make_pairs, axis=1)  
train.head(2)

,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,class_name,class_label,encode_fail,options
id,,,,,,,,,,,,
30192,gpt-4-1106-preview,gpt-4-0613,Is it morally right to try to have a certain p...,The question of whether it is morally right to...,"As an AI, I don't have personal beliefs or opi...",1,0,0,winner_model_a,0,False,[Prompt: Is it morally right to try to have a ...
53567,koala-13b,gpt-4-0613,What is the difference between marriage licens...,A marriage license is a legal document that al...,A marriage license and a marriage certificate ...,0,1,0,winner_model_b,1,False,[Prompt: What is the difference between marria...


In [8]:
print(train.encode_fail.value_counts(normalize=False))
train = train[train['encode_fail'] == False]
print(train.encode_fail.value_counts(normalize=False))


encode_fail
False    56885
True       592
Name: count, dtype: int64
encode_fail
False    56885
Name: count, dtype: int64


In [9]:
from sklearn.model_selection import train_test_split  # Import package

train = train.drop(['model_a', 'model_b', 'prompt', 'response_a','response_b', 'winner_model_a', 'winner_model_b', 'winner_tie', 'class_name'], axis=1)

train_df, valid_df = train_test_split(train, test_size=0.2, stratify=train["class_label"])

In [10]:
train_df.shape

(45508, 3)

In [11]:
from torch.utils.data import Dataset

class TextClassificationDataset(Dataset):
    def __init__(self, dataSet: pd.DataFrame, tokenizer):
        self.labels = dataSet['class_label'].values
        self.values = dataSet['options'].values
        self.dataSet = dataSet
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index):
        text = self.values[index]
        label = self.labels[index]
        encoding = self.tokenizer(
            text,
            max_length=256,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        input_ids = torch.reshape(encoding['input_ids'], (-1, ))
        attention_mask = torch.reshape(encoding['attention_mask'], (-1, ))
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "distilbert/distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3
)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

2025-05-14 09:42:55.322149: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747215775.562855      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747215775.634363      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
train_dataSet = TextClassificationDataset(train_df, tokenizer)
valid_dataSet = TextClassificationDataset(valid_df, tokenizer)

batch_size = 64
train_loader = DataLoader(train_dataSet, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataSet, batch_size=batch_size)

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
device

device(type='cuda')

In [15]:
from torch.optim import Adam

optimizer = Adam(model.parameters(), 2e-5)

In [17]:
num_epochs = 4
learning_rate = 2e-5

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    epoch_loss = 0
    model.train()

    for i, batch in enumerate(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}

        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        print(f" {i + 1}/{len(train_loader)}")
    avg_epoch_loss = epoch_loss / len(train_loader)
    print(f"Average Training Loss: {avg_epoch_loss:.4f}")

    valid_err = 0
    model.eval()
    with torch.no_grad():
        for i, batch in enumerate(valid_loader):
            batch = {k: v.to(device) for k, v in batch.items()}

            outputs = model(**batch)
            loss = outputs.loss
            valid_err += loss.item()
            print(f" {i + 1}/{len(valid_loader)}")
    avg_valid_loss = valid_err / len(valid_loader)
    print(f"Average Validation Loss: {avg_valid_loss:.4f}")



Epoch 1/4
 1/712
 2/712
 3/712
 4/712
 5/712
 6/712
 7/712
 8/712
 9/712
 10/712
 11/712
 12/712
 13/712
 14/712
 15/712
 16/712
 17/712
 18/712
 19/712
 20/712
 21/712
 22/712
 23/712
 24/712
 25/712
 26/712
 27/712
 28/712
 29/712
 30/712
 31/712
 32/712
 33/712
 34/712
 35/712
 36/712
 37/712
 38/712
 39/712
 40/712
 41/712
 42/712
 43/712
 44/712
 45/712
 46/712
 47/712
 48/712
 49/712
 50/712
 51/712
 52/712
 53/712
 54/712
 55/712
 56/712
 57/712
 58/712
 59/712
 60/712
 61/712
 62/712
 63/712
 64/712
 65/712
 66/712
 67/712
 68/712
 69/712
 70/712
 71/712
 72/712
 73/712
 74/712
 75/712
 76/712
 77/712
 78/712
 79/712
 80/712
 81/712
 82/712
 83/712
 84/712
 85/712
 86/712
 87/712
 88/712
 89/712
 90/712
 91/712
 92/712
 93/712
 94/712
 95/712
 96/712
 97/712
 98/712
 99/712
 100/712
 101/712
 102/712
 103/712
 104/712
 105/712
 106/712
 107/712
 108/712
 109/712
 110/712
 111/712
 112/712
 113/712
 114/712
 115/712
 116/712
 117/712
 118/712
 119/712
 120/712
 121/712
 122/712

In [18]:
torch.save(model.state_dict(), "model.pt")
torch.save(model, "model.h5")